## Topic Modeling of Lyrics with LDA

Author: Miles Mezaki

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/labeled_lyrics_cleaned.csv')
df

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371
...,...,...,...,...,...
158348,158348,Adam Green,"And we live on borrowed time,\r\nBut this head...",Friends of Mine,0.737
158349,158349,Adam Green,Frozin in time forever\r\nCarrying that torch ...,Frozen in Time,0.482
158350,158350,Adam Green,Hard to be a girl. \r\nSo nice to be a boy. \r...,Hard to Be a Girl,0.733
158351,158351,Adam Green,"I want to chose to die,\r\nAnd be buried with ...",I Wanna Die,0.361


Define code to expand contracted words, which are prominent in music:

In [3]:
def expand_contractions(s):
    res = ""
    contractions = { 
        "ain't": "is not", # Could be many tenses, but is not is probably the best mapping for all tenses
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "I'd": "I would",
        "I'd've": "I would have",
        "I'll": "I will",
        "I'll've": "I will have",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it would",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so is",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there would",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you would",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have"
        }
    for word in s.split():
        if word in contractions:
            res += contractions[word] + " "
        else:
            res += word + " "
    return res

print(df['seq'][0])

No, no
I ain't ever trapped out the bando
But oh Lord, don't get me wrong
I know a couple niggas that do
I'm from a place where everybody knows your name
They say I gotta watch my attitude
When they see money, man they all start actin' strange
So fuck with the ones that fuck with you
They can never say I'm brand new

It's everyday, everyday
Everyday, everyday, everyday
Everyday, everyday
Everyday, everyday
I've been talkin' my shit, nigga that's regular
I've been lovin' 'em thick, life is spectacular
I spend like I'ma die rich, nigga I'm flexin', yeah
Everyday, that's everyday
That's everyday
That's everyday
That's everyday, everyday

I see all of these wanna-be hot R&B singers
I swear you all sound the same
They start from the bottom, so far from the motto
You niggas'll never be Drake
Shout out to OVO
Most of them prolly don't know me though
I stay in the cut, I don't fuck with no
Body but I D, that's a pun on No I.D
When nobody know my name
Runnin' for my dream wasn't hard to do
You 

In [4]:
expand_contractions(df['seq'][0])

"No, no I is not ever trapped out the bando But oh Lord, do not get me wrong I know a couple niggas that do I am from a place where everybody knows your name They say I gotta watch my attitude When they see money, man they all start actin' strange So fuck with the ones that fuck with you They can never say I am brand new It's everyday, everyday Everyday, everyday, everyday Everyday, everyday Everyday, everyday I have been talkin' my shit, nigga that is regular I have been lovin' 'em thick, life is spectacular I spend like I'ma die rich, nigga I am flexin', yeah Everyday, that is everyday That's everyday That's everyday That's everyday, everyday I see all of these wanna-be hot R&B singers I swear you all sound the same They start from the bottom, so far from the motto You niggas'll never be Drake Shout out to OVO Most of them prolly do not know me though I stay in the cut, I do not fuck with no Body but I D, that is a pun on No I.D When nobody know my name Runnin' for my dream was not h

Next define code that will match expressions missing the trailing g in their progressive tense and replace the apostrophe with a g. (e.g. trippin' -> tripping)

In [5]:
import re

pattern = r"(.)in'"
replacement =  r"\1ing"
def expand_verbs(s: str, pattern, replacement):
    """
    Assume s is the lyrics string
    """
    res, _ = re.subn(pattern, replacement, s)
    return res

expand_verbs("trippin', grippin', dribblin'.", pattern, replacement)

'tripping, gripping, dribbling.'

In [6]:
def expand_verbs_and_contractions(s, pattern=r"(.)in'", replacement=r"\1ing"):
    """
    s: string representing lyrics
    pattern: regular expression representing the pattern to be replaced
    replacement: regular expression to replace pattern

    Returns:
        res: string ready for tokenization
    """
    return expand_verbs(expand_contractions(s.lower()), pattern, replacement)
    
expand_verbs_and_contractions(df['seq'][0])

"no, no i is not ever trapped out the bando but oh lord, do not get me wrong i know a couple niggas that do i'm from a place where everybody knows your name they say i gotta watch my attitude when they see money, man they all start acting strange so fuck with the ones that fuck with you they can never say i'm brand new it is everyday, everyday everyday, everyday, everyday everyday, everyday everyday, everyday i've been talking my shit, nigga that is regular i've been loving 'em thick, life is spectacular i spend like i'ma die rich, nigga i'm flexing, yeah everyday, that is everyday that is everyday that is everyday that is everyday, everyday i see all of these wanna-be hot r&b singers i swear you all sound the same they start from the bottom, so far from the motto you niggas'll never be drake shout out to ovo most of them prolly do not know me though i stay in the cut, i do not fuck with no body but i d, that is a pun on no i.d when nobody know my name running for my dream was not hard

In [7]:
df['seq'] = df['seq'].apply(expand_verbs_and_contractions)

In [8]:
df.head()

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"no, no i is not ever trapped out the bando but...",Everyday,0.626
1,1,Elijah Blake,"the drinks go down and smoke goes up, i feel m...",Live Till We Die,0.630
2,2,Elijah Blake,she do not live on planet earth no more she fo...,The Otherside,0.240
3,3,Elijah Blake,"tripping off that grigio, mobbing, lights low ...",Pinot,0.536
4,4,Elijah Blake,"i see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371


Next, we want to tokenize and stem the data. We will not remove stop words since the people are generally important in lyrics and the verbs are often stopwords.

In [9]:
from nltk.tokenize import word_tokenize 

text = df['seq'][0]
# tokenize
result = word_tokenize(text)
result

['no',
 ',',
 'no',
 'i',
 'is',
 'not',
 'ever',
 'trapped',
 'out',
 'the',
 'bando',
 'but',
 'oh',
 'lord',
 ',',
 'do',
 'not',
 'get',
 'me',
 'wrong',
 'i',
 'know',
 'a',
 'couple',
 'niggas',
 'that',
 'do',
 'i',
 "'m",
 'from',
 'a',
 'place',
 'where',
 'everybody',
 'knows',
 'your',
 'name',
 'they',
 'say',
 'i',
 'got',
 'ta',
 'watch',
 'my',
 'attitude',
 'when',
 'they',
 'see',
 'money',
 ',',
 'man',
 'they',
 'all',
 'start',
 'acting',
 'strange',
 'so',
 'fuck',
 'with',
 'the',
 'ones',
 'that',
 'fuck',
 'with',
 'you',
 'they',
 'can',
 'never',
 'say',
 'i',
 "'m",
 'brand',
 'new',
 'it',
 'is',
 'everyday',
 ',',
 'everyday',
 'everyday',
 ',',
 'everyday',
 ',',
 'everyday',
 'everyday',
 ',',
 'everyday',
 'everyday',
 ',',
 'everyday',
 'i',
 "'ve",
 'been',
 'talking',
 'my',
 'shit',
 ',',
 'nigga',
 'that',
 'is',
 'regular',
 'i',
 "'ve",
 'been',
 'loving',
 "'em",
 'thick',
 ',',
 'life',
 'is',
 'spectacular',
 'i',
 'spend',
 'like',
 "i'ma",
 '

In [17]:
from nltk.stem import PorterStemmer
 
ps = PorterStemmer()
 
# choose some words to be stemmed
words = df['seq'][3]
 
for w in words.split():
    print(w, " : ", ps.stem(w))

tripping  :  trip
off  :  off
that  :  that
grigio,  :  grigio,
mobbing,  :  mobbing,
lights  :  light
low  :  low
tripping  :  trip
off  :  off
that  :  that
grigio,  :  grigio,
mobbing,  :  mobbing,
lights  :  light
low  :  low
tripping  :  trip
off  :  off
that  :  that
grigio,  :  grigio,
mobbing,  :  mobbing,
lights  :  light
low  :  low
baby  :  babi
i'm  :  i'm
surprised  :  surpris
you  :  you
picked  :  pick
up  :  up
this  :  thi
late  :  late
at  :  at
night  :  night
and  :  and
i'm  :  i'm
not  :  not
tryna  :  tryna
start  :  start
a  :  a
fight  :  fight
but  :  but
i  :  i
hate  :  hate
the  :  the
way  :  way
we  :  we
ended  :  end
and  :  and
the  :  the
fact  :  fact
you  :  you
got  :  got
me  :  me
cheated  :  cheat
i  :  i
hate  :  hate
these  :  these
new  :  new
beginnings  :  begin
even  :  even
though  :  though
i  :  i
am  :  am
the  :  the
reason  :  reason
the  :  the
only  :  onli
thing  :  thing
i  :  i
love  :  love
right  :  right
now  :  now
is  :  is

In [18]:
def token_stem(s):
    """
    Given a string s, tokenize the string, then stem it.
    """
    ps = PorterStemmer()
    word_tokenize(s)
    return " ".join([ps.stem(w) for w in word_tokenize(s)])

token_stem("converts the words in word_tokens to lower case and then checks whether ")

'convert the word in word_token to lower case and then check whether'

Now we apply the tokenization and stemming to our lyrics.

In [19]:
df['seq'] = df['seq'].apply(token_stem)

Now our lyrics text is ready for post-processing.

In [20]:
df['seq']

0         no , no i is not ever trap out the bando but o...
1         the drink go down and smoke goe up , i feel my...
2         she do not live on planet earth no more she fo...
3         trip off that grigio , mob , light low trip of...
4         i see a midnight panther , so gallant and so b...
                                ...                        
158348    and we live on borrow time , but thi headshot ...
158349    frozin in time forev carri that torch for so l...
158350    hard to be a girl . so nice to be a boy . in m...
158351    i want to chose to die , and be buri with a ru...
158352    music ladder lean on mountain bath in white la...
Name: seq, Length: 158353, dtype: object

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
# Initialize the vectorizer
vectorizer = CountVectorizer(
    strip_accents='unicode',
    stop_words='english',
    lowercase=True,
    token_pattern=r'\b[a-zA-Z]{3,}\b', # we want only words that contain letters and are 3 or more characters long
)

# Transform our data into the document-term matrix
dtm = vectorizer.fit_transform(df['seq'])
dtm

<158353x99073 sparse matrix of type '<class 'numpy.int64'>'
	with 8188414 stored elements in Compressed Sparse Row format>

In [23]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['aaa', 'aaaa', 'aaaaa', ..., 'zzznoahh', 'zzzz', 'zzzzzombiee'],
      dtype=object)

In [24]:
feature_names.shape

(99073,)

In [25]:
feature_names[300:350]

array(['abgefickt', 'abgehn', 'abgesaugt', 'abgestellt', 'abgrund',
       'abgvll', 'abh', 'abhaya', 'abhor', 'abhorr', 'abi', 'abid',
       'abidin', 'abierta', 'abierto', 'abigail', 'abil', 'abilen',
       'abilin', 'abillatoeya', 'abillybopa', 'abilti', 'abireru',
       'abismo', 'abit', 'abita', 'abitacion', 'abitai', 'abitar',
       'abitch', 'abito', 'abituati', 'abitudin', 'abiyoyo', 'abizov',
       'abject', 'abjur', 'abk', 'abl', 'ablaz', 'able', 'ablebodi',
       'ableton', 'abli', 'ablig', 'ablow', 'ablut', 'abn', 'abneg',
       'abner'], dtype=object)

In [26]:
doc1 = dtm[0]
doc1

<1x99073 sparse matrix of type '<class 'numpy.int64'>'
	with 70 stored elements in Compressed Sparse Row format>

In [27]:
row_index = 0
doc_vec = dtm.getrow(row_index).toarray()

non_zero_indices = doc_vec.nonzero()[1]
dtm_scores = doc_vec[0, non_zero_indices] # goes and retrieves the values corresponding to the non_zero_indices
words = [feature_names[i] for i in non_zero_indices]

for word, score in zip(words, dtm_scores):
    print(f"{word}: {score}")

act: 2
attitud: 1
bando: 1
bodi: 1
brand: 1
bread: 1
break: 1
chang: 2
coupl: 1
crew: 1
cut: 1
die: 3
drake: 1
dream: 1
dress: 1
eat: 1
everybodi: 1
everyday: 45
far: 1
flex: 3
fuck: 3
funni: 1
got: 3
hard: 1
heard: 1
hobbi: 1
hot: 1
know: 4
life: 3
like: 3
lone: 1
lord: 1
lose: 1
love: 4
man: 1
money: 2
motto: 1
new: 1
nigga: 8
nobodi: 2
ovo: 1
place: 1
plate: 1
prolli: 1
pull: 1
pun: 1
regular: 3
rich: 3
run: 1
say: 2
shit: 3
shout: 1
sinc: 1
singer: 1
sound: 1
spectacular: 3
spend: 3
start: 2
starv: 1
stay: 1
strang: 1
swear: 2
talk: 3
trap: 1
wanna: 1
watch: 1
whi: 1
win: 1
wrong: 1
yeah: 3


In [28]:
non_zero_indices

array([  778,  5043,  6389,  9715, 10697, 10810, 10823, 14424, 18595,
       19060, 19896, 22649, 24611, 24682, 24761, 25673, 28121, 28127,
       29114, 30628, 32182, 32431, 35053, 37188, 37767, 39125, 39814,
       46577, 49098, 49218, 49991, 50168, 50210, 50314, 51716, 55677,
       56316, 58451, 58716, 59106, 62116, 65318, 65451, 67505, 68061,
       68117, 70432, 71538, 73121, 74428, 77241, 77539, 78176, 78207,
       80507, 80842, 80907, 81924, 81946, 82023, 82694, 84406, 85063,
       88768, 94700, 94947, 95752, 96322, 97219, 97727])

In [29]:
dtm.getcol(2327).toarray().T # get the column, turn it into an array format, then transpose it to be a row

array([[0, 0, 0, ..., 0, 0, 0]])

In [30]:
dtm.getcol(44592).toarray().T # Fuck, which should appear often in songs

array([[0, 0, 0, ..., 0, 0, 0]])

In [31]:
np.count_nonzero(dtm.getcol(44592).toarray().T)

1

Back to DF

In [32]:
def matrix2Doc(dtMatrix, features, index):
    """Turns each row of the document-term matrix into a list of terms"""
    row = dtMatrix.getrow(index).toarray()
    non_zero_indices = row.nonzero()[1]
    words = [features[idx] for idx in non_zero_indices]
    return words

In [33]:
allDocsAsTerms = [matrix2Doc(dtm, feature_names, i) for i in range(dtm.shape[0])]

In [34]:
len(allDocsAsTerms)

158353

In [35]:
df['terms'] = allDocsAsTerms
df.head()

,Unnamed: 0,artist,seq,song,label,terms
0,0,Elijah Blake,"no , no i is not ever trap out the bando but o...",Everyday,0.626,"[act, attitud, bando, bodi, brand, bread, brea..."
1,1,Elijah Blake,"the drink go down and smoke goe up , i feel my...",Live Till We Die,0.630,"[away, band, bite, blow, care, chase, circl, c..."
2,2,Elijah Blake,she do not live on planet earth no more she fo...,The Otherside,0.240,"[bad, bag, befor, broke, broken, caught, cheat..."
3,3,Elijah Blake,"trip off that grigio , mob , light low trip of...",Pinot,0.536,"[alway, babi, becaus, begin, blow, boy, cheat,..."
4,4,Elijah Blake,"i see a midnight panther , so gallant and so b...",Shadows & Diamonds,0.371,"[alon, answer, believ, brave, broke, coal, cof..."


In [36]:
from sklearn.decomposition import LatentDirichletAllocation

# Step 1: Initialize the model

lda = LatentDirichletAllocation(n_components=15, # we are picking the number of topics arbitrarely at the moment
                                random_state=0)

# Step 2: Fit the model
lda.fit(dtm)

LatentDirichletAllocation(n_components=15, random_state=0)

In [37]:
lda.components_

array([[9.68600788, 0.06666695, 0.06666748, ..., 0.06666667, 0.06666667,
        0.06666667],
       [0.066667  , 0.06666725, 0.06666667, ..., 0.06666682, 0.06666725,
        0.06666695],
       [0.06666683, 1.99695389, 0.06666672, ..., 0.06666667, 0.06666667,
        0.06666667],
       ...,
       [0.06666678, 0.06666674, 0.06666667, ..., 0.06666667, 0.06666669,
        0.06666671],
       [0.06666672, 1.37573704, 0.06666667, ..., 0.06666667, 0.06666675,
        1.06666115],
       [6.21954668, 1.85231007, 0.06666667, ..., 0.06666667, 2.066665  ,
        0.06666728]])

In [39]:
lda.components_.shape

(15, 99073)

In [40]:
def display_topics(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")
        print(" ".join([features[i]
                        for i in topic.argsort()[:-no_top_words-1:-1]])) # syntax for reversing a list [::-1]

display_topics(lda, feature_names, 10)

Topic 0:
pum doo dem mari boom beat yuh rum nah america
Topic 1:
nigga fuck got like shit bitch thi money know ass
Topic 2:
lord good ride tonight gon jesu alright sing king got
Topic 3:
rock got new like work just play roll thi bye
Topic 4:
come day away home night time sun walk littl morn
Topic 5:
love dream heart song sing long old just night kiss
Topic 6:
blue cri fli high sky whi lone mama die happi
Topic 7:
run que wild ladi devil whi sha tiger amor soul
Topic 8:
like just onli believ make thing said head smile eye
Topic 9:
love babi want know let just feel yeah need got
Topic 10:
know time just thi say think feel tri way want
Topic 11:
yeah got girl like gon know let come danc littl
Topic 12:
thi let god eye soul burn life come feel light
Topic 13:
world live life hey thi look way peopl turn round
Topic 14:
man like woman got gon gun just kid said run


In [41]:
doc_topic_dist = lda.transform(dtm)
doc_topic_dist 

array([[4.14078826e-04, 7.14970740e-01, 4.14081531e-04, ...,
        4.14079168e-04, 7.36064008e-02, 4.14079439e-04],
       [3.00300487e-04, 8.21262441e-02, 3.00300738e-04, ...,
        3.00301014e-04, 5.05506867e-01, 3.00300814e-04],
       [6.80272233e-04, 1.73118525e-01, 6.80272906e-04, ...,
        6.80273685e-04, 1.06140534e-01, 6.80274272e-04],
       ...,
       [1.75438686e-03, 1.75438899e-03, 4.48299426e-02, ...,
        1.75439384e-03, 1.75438897e-03, 1.08689853e-01],
       [1.23456949e-03, 1.70729568e-01, 1.23457064e-03, ...,
        1.92636023e-01, 1.23457129e-03, 2.84377936e-01],
       [1.19047677e-03, 1.19047998e-03, 9.44322446e-02, ...,
        1.19048014e-03, 6.72087154e-02, 2.26268293e-01]])

In [42]:
doc_topic_dist.shape

(158353, 15)

In [43]:
def displayHeader(model, features, no_top_words):
    """Helper function to show the top words of a model"""
    topicNames = []
    for topic_idx, topic in enumerate(model.components_):
        topicNames.append(f"Topic {topic_idx}: " + (", ".join([features[i]
                             for i in topic.argsort()[:-no_top_words-1:-1]])))
    return topicNames

In [44]:
# column names
topicnames = displayHeader(lda, feature_names, 5)

# index names
docnames = df.index.tolist() # We will use the original names of the documents

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(doc_topic_dist, 3), 
                                 columns=topicnames, 
                                 index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1) # finds the maximum argument
df_document_topic['dominant_topic'] = dominant_topic

df_document_topic.head()

,"Topic 0: pum, doo, dem, mari, boom","Topic 1: nigga, fuck, got, like, shit","Topic 2: lord, good, ride, tonight, gon","Topic 3: rock, got, new, like, work","Topic 4: come, day, away, home, night","Topic 5: love, dream, heart, song, sing","Topic 6: blue, cri, fli, high, sky","Topic 7: run, que, wild, ladi, devil","Topic 8: like, just, onli, believ, make","Topic 9: love, babi, want, know, let","Topic 10: know, time, just, thi, say","Topic 11: yeah, got, girl, like, gon","Topic 12: thi, let, god, eye, soul","Topic 13: world, live, life, hey, thi","Topic 14: man, like, woman, got, gon",dominant_topic
0,0.000,0.715,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.117,0.090,0.000,0.000,0.074,0.000,1
1,0.000,0.082,0.000,0.000,0.000,0.000,0.246,0.000,0.000,0.000,0.000,0.163,0.000,0.506,0.000,13
2,0.001,0.173,0.001,0.001,0.001,0.457,0.001,0.001,0.046,0.001,0.091,0.120,0.001,0.106,0.001,5
3,0.000,0.237,0.000,0.000,0.057,0.000,0.000,0.000,0.000,0.089,0.364,0.249,0.000,0.000,0.000,10
4,0.111,0.001,0.001,0.001,0.001,0.026,0.054,0.001,0.001,0.299,0.186,0.001,0.143,0.001,0.172,9


In [45]:
df_document_topic[76:86]

,"Topic 0: pum, doo, dem, mari, boom","Topic 1: nigga, fuck, got, like, shit","Topic 2: lord, good, ride, tonight, gon","Topic 3: rock, got, new, like, work","Topic 4: come, day, away, home, night","Topic 5: love, dream, heart, song, sing","Topic 6: blue, cri, fli, high, sky","Topic 7: run, que, wild, ladi, devil","Topic 8: like, just, onli, believ, make","Topic 9: love, babi, want, know, let","Topic 10: know, time, just, thi, say","Topic 11: yeah, got, girl, like, gon","Topic 12: thi, let, god, eye, soul","Topic 13: world, live, life, hey, thi","Topic 14: man, like, woman, got, gon",dominant_topic
76,0.001,0.036,0.028,0.558,0.001,0.001,0.001,0.040,0.001,0.001,0.260,0.001,0.073,0.001,0.001,3
77,0.001,0.001,0.001,0.077,0.001,0.110,0.001,0.001,0.001,0.001,0.595,0.198,0.001,0.015,0.001,10
78,0.001,0.250,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.166,0.574,0.001,0.001,0.001,11
79,0.001,0.001,0.001,0.001,0.050,0.178,0.065,0.001,0.001,0.119,0.579,0.001,0.001,0.001,0.001,10
80,0.001,0.001,0.001,0.001,0.027,0.001,0.001,0.001,0.001,0.040,0.131,0.163,0.001,0.633,0.001,13
81,0.000,0.243,0.000,0.191,0.000,0.000,0.000,0.000,0.047,0.402,0.000,0.000,0.000,0.000,0.112,9
82,0.000,0.000,0.000,0.560,0.000,0.159,0.000,0.000,0.115,0.000,0.000,0.162,0.000,0.000,0.000,3
83,0.000,0.000,0.033,0.000,0.024,0.137,0.048,0.033,0.000,0.000,0.098,0.290,0.235,0.000,0.100,11
84,0.001,0.330,0.001,0.001,0.001,0.001,0.001,0.001,0.072,0.001,0.542,0.049,0.001,0.001,0.001,10
85,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.759,0.142,0.001,0.091,0.001,0.001,9


In [46]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,10,43417
1,12,21960
2,9,17647
3,5,12412
4,4,11316
5,11,8757
6,3,8721
7,14,7296
8,1,6983
9,8,6087


In [47]:
df.to_csv('data/stemmed_cleaned_lyrics.csv')
df_document_topic.to_csv('lda-results/document_topic.csv')
df_topic_distribution.to_csv('lda-results/topic_distribution.csv')

In [46]:
from sklearn.model_selection import GridSearchCV

# We are going to test multiple values for the number of topics
search_params = {'n_components': [5, 10, 15, 20, 25, 30, 35]}

# Initialize the LDA model
lda = LatentDirichletAllocation()

# Initialize a Grid Search with cross-validation instance
grid = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
grid.fit(dtm)

KeyboardInterrupt: 

In [ ]:
grid.cv_results_

In [ ]:
# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

In [ ]:
search_params = {'n_components': [1,2,3,4,5,6]}

lda = LatentDirichletAllocation()
grid = GridSearchCV(lda, param_grid=search_params)

grid.fit(dtm)

# Best Model
best_lda_model = grid.best_estimator_

# Model Parameters
print("Best Model's Params: ", grid.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", grid.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

In [ ]:
display_topics(best_lda_model, feature_names, 40)